In [21]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [22]:
%pip install numpy pygame tqdm matplotlib ipywidgets pandas torch Pillow ipympl
%pip install minedojo --user

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


#Setting up PyTorch for GPU

In [23]:
import torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical

torch.cuda.is_available()

True

In [24]:
import numpy as np
import itertools
from tqdm.notebook import trange, tqdm
import pandas as pd
import sys
from abc import ABC
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib.pyplot as plt
from PIL import Image
import minedojo
%matplotlib widget

In [25]:
minedojo.tasks.ALL_PROGRAMMATIC_TASK_IDS

['combat_shulker_end_leather_armors_wooden_sword_shield',
 'combat_shulker_end_iron_armors_wooden_sword_shield',
 'combat_shulker_end_diamond_armors_wooden_sword_shield',
 'combat_endermite_end_leather_armors_wooden_sword_shield',
 'combat_endermite_end_iron_armors_wooden_sword_shield',
 'combat_endermite_end_diamond_armors_wooden_sword_shield',
 'combat_shulker_end_leather_armors_iron_sword_shield',
 'combat_shulker_end_iron_armors_iron_sword_shield',
 'combat_shulker_end_diamond_armors_iron_sword_shield',
 'combat_endermite_end_leather_armors_iron_sword_shield',
 'combat_endermite_end_iron_armors_iron_sword_shield',
 'combat_endermite_end_diamond_armors_iron_sword_shield',
 'combat_shulker_end_leather_armors_diamond_sword_shield',
 'combat_shulker_end_iron_armors_diamond_sword_shield',
 'combat_shulker_end_diamond_armors_diamond_sword_shield',
 'combat_endermite_end_leather_armors_diamond_sword_shield',
 'combat_endermite_end_iron_armors_diamond_sword_shield',
 'combat_endermite_end_

In [26]:
from helpers.hunt_cow import HuntCowDenseRewardEnv

# IMAGE_SIZE = (142,255)
IMAGE_SIZE = (160,256)
# env = minedojo.make("harvest_wool_with_shears_and_sheep", image_size=(160, 256) )
env = HuntCowDenseRewardEnv(
    step_penalty=0.1,
    nav_reward_scale=2,
    attack_reward=10,
    success_reward=100,
    image_size=IMAGE_SIZE,
)
env.task_prompt

<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
/home/sasha/.local/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


'Hunt 1 pig, 1 cow, 1 sheep.'

In [27]:
# Set the extra environment variables
# env.set_env_var("MINEDOJO_HEADLESS", "1")

In [28]:
#https://docs.minedojo.org/sections/core_api/action_space.html
act_space = env.action_space

In [29]:
obs_space = env.observation_space

In [30]:
import csv

def load_item_mapping_from_csv(file_path):
    item_mapping = {}
    with open(file_path, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip the header row if it exists
        for row in reader:
            item_name, _, item_id, item_id_2 = row  # Change this line to read the first and third columns
            if item_id is None or item_id == '':
                item_id = 10000 + len(item_mapping)
            item_id = int(item_id)
            if item_id_2 is not None:
                item_id = item_id + 10000 * int(item_id_2)

            item_mapping[item_name] = item_id
            item_mapping[item_name.replace(" ", "_")] = item_id
            item_mapping[item_name.replace(" ", "_").lower()] = item_id
            item_mapping[item_name.replace(" ", "_").upper()] = item_id
            item_mapping[item_name.replace(" ", "_").capitalize()] = item_id
            item_mapping[item_name.replace(" ", "_").lower().capitalize()] = item_id
            item_mapping[item_name.replace(" ", "_").upper().capitalize()] = item_id
            item_mapping[item_id] = item_id
    return item_mapping


def minimizeIds(item_mapping):
    # minimizing IDs by replacing each items id with its index in the list
    # this is done to reduce the size of the observation space
    minimized = {}
    for key, value in item_mapping.items():
        if key not in minimized:
            minimized[key] = len(minimized)
    return minimized

# ensure all numeric ids are unique
def check_item_mapping(item_mapping):
    numeric_ids = [item_id for item_id in item_mapping.values() if isinstance(item_id, int)]
    assert len(numeric_ids) == len(set(numeric_ids))

string_to_index_mapping = load_item_mapping_from_csv("minecraft_items.csv")

string_to_index_mapping = minimizeIds(string_to_index_mapping)

check_item_mapping(string_to_index_mapping)

string_to_index_mapping

{'acacia door': 0,
 'acacia_door': 1,
 'ACACIA_DOOR': 2,
 'Acacia_door': 3,
 430: 4,
 'acacia fence': 5,
 'acacia_fence': 6,
 'ACACIA_FENCE': 7,
 'Acacia_fence': 8,
 192: 9,
 'acacia fence gate': 10,
 'acacia_fence_gate': 11,
 'ACACIA_FENCE_GATE': 12,
 'Acacia_fence_gate': 13,
 187: 14,
 'acacia stairs': 15,
 'acacia_stairs': 16,
 'ACACIA_STAIRS': 17,
 'Acacia_stairs': 18,
 163: 19,
 'activator rails': 20,
 'activator_rails': 21,
 'ACTIVATOR_RAILS': 22,
 'Activator_rails': 23,
 157: 24,
 'air': 25,
 'AIR': 26,
 'Air': 27,
 0: 28,
 'anvil': 29,
 'ANVIL': 30,
 'Anvil': 31,
 145: 32,
 'slightly damaged anvil': 33,
 'slightly_damaged_anvil': 34,
 'SLIGHTLY_DAMAGED_ANVIL': 35,
 'Slightly_damaged_anvil': 36,
 10145: 37,
 'very damaged anvil': 38,
 'very_damaged_anvil': 39,
 'VERY_DAMAGED_ANVIL': 40,
 'Very_damaged_anvil': 41,
 20145: 42,
 'apple': 43,
 'APPLE': 44,
 'Apple': 45,
 260: 46,
 'armor stand': 47,
 'armor_stand': 48,
 'ARMOR_STAND': 49,
 'Armor_stand': 50,
 416: 51,
 'arrow': 52,


In [31]:
import gymnasium as gym

def preprocess_observation(obs_dict, space_dict = obs_space, string_to_index_mapping=string_to_index_mapping):
    rgb = obs_dict['rgb']
    equipment = obs_dict['equipment']["name"]
    equipment_count = obs_dict['equipment']["quantity"]
    inventory = obs_dict['inventory']['name']
    inventory_count = obs_dict['inventory']['quantity']
    block_names = obs_dict["voxels"]["block_name"]
    block_collidable = obs_dict["voxels"]["is_collidable"]
    tool_req = obs_dict["voxels"]["is_tool_not_required"]
    is_liquid = obs_dict["voxels"]["is_liquid"]
    is_solid = obs_dict["voxels"]["is_solid"]
    look_vector = obs_dict["voxels"]["cos_look_vec_angle"]
    cur_durability = obs_dict["equipment"]["cur_durability"]
    max_durability = obs_dict["equipment"]["max_durability"]
    inc_name_by_craft = obs_dict["delta_inv"]["inc_name_by_craft"]
    inc_quantity_by_craft = obs_dict["delta_inv"]["inc_quantity_by_craft"]
    inc_name_by_other = obs_dict["delta_inv"]["inc_name_by_other"]
    inc_quantity_by_other = obs_dict["delta_inv"]["inc_quantity_by_other"]
    dec_name_by_craft = obs_dict["delta_inv"]["dec_name_by_craft"]
    dec_quantity_by_craft = obs_dict["delta_inv"]["dec_quantity_by_craft"]
    dec_name_by_other = obs_dict["delta_inv"]["dec_name_by_other"]
    dec_quantity_by_other = obs_dict["delta_inv"]["dec_quantity_by_other"]
    life = obs_dict["life_stats"]["life"]
    oxygen = obs_dict["life_stats"]["oxygen"]
    armor = obs_dict["life_stats"]["armor"]
    food = obs_dict["life_stats"]["food"]
    saturation = obs_dict["life_stats"]["saturation"]
    is_sleeping = obs_dict["life_stats"]["is_sleeping"]
    xp = obs_dict["life_stats"]["xp"]
    pos = obs_dict["location_stats"]["pos"]
    damage_amount = obs_dict["damage_source"]["damage_amount"]

    # give us a 1d array of all this stuff smooshed into ints
    obs_list = []

    rgb_flattened = np.zeros((len(rgb[0]), len(rgb[0][0])), dtype=np.uint32)
    for channel in rgb:
        for idx, row in enumerate(channel):
            for idx2, item in enumerate(row):
                rgb_flattened[idx][idx2] = rgb_flattened[idx][idx2] << 8
                rgb_flattened[idx][idx2] = rgb_flattened[idx][idx2] | item


    for row in rgb_flattened:
        for item in row:
            obs_list.append(item)
    for item in equipment:
        obs_list.append(string_to_index_mapping[item])
    for item in equipment_count:
        obs_list.append(item)
    for item in inventory:
        obs_list.append(string_to_index_mapping[item])
    for item in inventory_count:
        obs_list.append(item)
    for list in block_names:
        for sub_list in list:
            for item in sub_list:
                obs_list.append(string_to_index_mapping[item])
    block_collidable_int = 0
    for item in block_collidable:
        # 2 nested lists inside this, but we want to flatten them each to ints using bit shifting
        for sub_list in item:
            for sub_item in sub_list:
                block_collidable_int = block_collidable_int << 1
                block_collidable_int = block_collidable_int | sub_item
    obs_list.append(block_collidable_int)
    tool_req_int = 0
    for item in tool_req:
        # 2 nested lists inside this, but we want to flatten them each to ints using bit shifting
        for sub_list in item:
            for sub_item in sub_list:
                tool_req_int = tool_req_int << 1
                tool_req_int = tool_req_int | sub_item
    obs_list.append(tool_req_int)
    is_liquid_int = 0
    for item in is_liquid:
        # 2 nested lists inside this, but we want to flatten them each to ints using bit shifting
        for sub_list in item:
            for sub_item in sub_list:
                is_liquid_int = is_liquid_int << 1
                is_liquid_int = is_liquid_int | sub_item
    obs_list.append(is_liquid_int)
    is_solid_int = 0
    for item in is_solid:
        # 2 nested lists inside this, but we want to flatten them each to ints using bit shifting
        for sub_list in item:
            for sub_item in sub_list:
                is_solid_int = is_solid_int << 1
                is_solid_int = is_solid_int | sub_item
    obs_list.append(is_solid_int)
    for item in look_vector:
        for sub_item in item:
            for sub_sub_item in sub_item:
                obs_list.append(sub_sub_item)

    #Data type: numpy.float32
    # Shape: (6,)
    for item in cur_durability:
        obs_list.append(item)

    for item in max_durability:
        obs_list.append(item)

    for item in inc_name_by_craft:
        obs_list.append(string_to_index_mapping[item])
    for item in inc_quantity_by_craft:
        obs_list.append(item)

    for item in inc_name_by_other:
        obs_list.append(string_to_index_mapping[item])
    for item in inc_quantity_by_other:
        obs_list.append(item)

    for item in dec_name_by_craft:
        obs_list.append(string_to_index_mapping[item])
    for item in dec_quantity_by_craft:
        obs_list.append(item)

    for item in dec_name_by_other:
        obs_list.append(string_to_index_mapping[item])
    for item in dec_quantity_by_other:
        obs_list.append(item)

    for item in life:
        obs_list.append(item)

    for item in oxygen:
        obs_list.append(item)

    for item in armor:
        obs_list.append(item)

    for item in food:
        obs_list.append(item)

    for item in saturation:
        obs_list.append(item)

    obs_list.append(is_sleeping)

    # for item in xp:
    #     obs_list.append(item)
    #
    # #
    # for item in pos:
    #     obs_list.append(item)
    #
    # obs_list.append(damage_amount)



    return np.array(obs_list, dtype=np.float32)



In [ ]:
input_shape = preprocess_observation(env.reset()).shape[0]
num_actions = env.action_space.nvec.tolist()
print(f"Input shape: {input_shape} | Number of actions: {num_actions}")
#
# import gc
# gc.collect()
# torch.cuda.empty_cache()


In [ ]:
# # todo: Masks are still not being applied properly. The agent is masking too much and can only ever look up. Peek `training.log`
# def mask_multidiscrete_probs(masks, multidiscrete_tensor):
#     # detach all tensors from the graph
#     multidiscrete_tensor = [t.detach().cpu() for t in multidiscrete_tensor]
#     # Create a list of zero-filled tensors with the same shapes as multidiscrete_tensor
#     result = [torch.zeros_like(t) for t in multidiscrete_tensor]

#     for i, key in enumerate(masks.keys()):
#         mask = masks[key]
#         for j, is_enabled in enumerate(mask) :
#             if is_enabled and i < len(multidiscrete_tensor) and j < len(multidiscrete_tensor[i]):
#                 result[i][j] = multidiscrete_tensor[i][j]

#     return result


def get_args_mask_for_func_action(fun_act_idx) -> str:
    if(fun_act_idx == 0):
        return 'no-op'
    elif(fun_act_idx == 1):
        return 'use'
    elif(fun_act_idx == 2):
        return 'drop'
    elif(fun_act_idx == 3):
        return 'attack'
    elif(fun_act_idx == 4):
        return 'craft_smelt'
    elif(fun_act_idx == 5):
        return 'equip'
    elif(fun_act_idx == 6):
        return 'place'
    elif(fun_act_idx == 7):
        return 'destroy'
    else:
        return 'no-op'


def any_valid_arg(mask):
    # return true if any bool in mask is true, mask is 1d
    return np.any(mask)

def mask_apply(masks, multidiscrete_tensor, isRandom=False, returnProbs=False):
    """
             apply masks as follows:
              1. Determine functional action (index 5, mask 'action_type') (size 8 bools)
                 - action for subsequent steps is argmax of non-false masked values
              2. Determine if action takes args (index 5, mask 'action_arg') (size 8 bools)
              3. Based on action type, pick the corresponding mask from:
                 - 'equip'      (size 36 bools, corresponds to inventory slots)
                 - 'place'      (size 36 bools, corresponds to inventory slots)
                 - 'destroy'    (size 36 bools, corresponds to inventory slots)
                 - 'craft_smelt'    (size 244 bools, corresponds to all craftable items)
                 - no mask for other options
    :param masks: a dict of np array of bools with keys corresponding to what is being masked, as described above
    :param multidiscrete_tensor: list of tensors of probabilities for every possible action
    :return: the choice made based on argmax applied to each tensor in multidiscrete_tensor after masking
    """
    # detach all tensors from the graph
    multidiscrete_tensor = [t.detach().cpu() for t in multidiscrete_tensor]
    # Create a list of zero-filled tensors with the same shapes as multidiscrete_tensor
    result = [torch.zeros_like(t) for t in multidiscrete_tensor]

    for(i, t) in enumerate(multidiscrete_tensor):
        for(j, p) in enumerate(t):
            result[i][j] = p

    # Apply func mask to index 5
    func_mask = masks['action_type']
    print(func_mask)
    for i, is_enabled in enumerate(func_mask):
        if is_enabled and i < len(multidiscrete_tensor[5]):
            if i > 3 and any_valid_arg(masks[get_args_mask_for_func_action(i)]):
                result[5][i] = multidiscrete_tensor[5][i] if not isRandom else np.random.uniform() + 0.5
            elif i <= 3:
                result[5][i] = multidiscrete_tensor[5][i] if not isRandom else np.random.uniform() + 0.5
    print(len(result[5]))
    print(result[5])

    func_act = torch.argmax(result[5])
    print(f"func_act: {func_act}")
    # check arg mask with the choice
    args_mask = masks['action_arg']
    print(args_mask)
    if(args_mask[func_act.item()] == True):
        msk = masks[get_args_mask_for_func_action(func_act.item())]
        idx_of_result_to_use = 6 if  get_args_mask_for_func_action(func_act.item()) == 'craft_smelt' else 7
        for i, is_enabled in enumerate(msk):
            if i < len(multidiscrete_tensor[idx_of_result_to_use]):
                if is_enabled:
                    result[idx_of_result_to_use][i] = multidiscrete_tensor[idx_of_result_to_use][i] if not isRandom else np.random.uniform() + 0.5
                else:
                    result[idx_of_result_to_use][i] = 0

    # argmax all the things
    if returnProbs:
        return result
    for i, t in enumerate(result):
        result[i] = torch.argmax(t).item()
    logging.debug(f"Masked result: {result}")
    return result

In [ ]:
import math


class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dims):
        super(ActorCritic, self).__init__()
        self.device = torch.device("cpu")
        # torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.actor_branches = nn.ModuleList().to(self.device)
        # Set the desired dimensions for reshaping

        self.C, self.H, self.W = 1, 187, 220

        self.actor_branches = nn.ModuleList().to(self.device)
        for action_dim in action_dims:
            self.actor_branches.append(nn.Sequential(
                nn.Conv2d(self.C, 32, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2),
                nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2),
                nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2),
                nn.Flatten(),
                nn.Flatten(),
                nn.Linear(621, 256),
                nn.ReLU(),
                nn.Linear(256, action_dim),
                # dimension is now (128, action_dim). We want to have (action_dim).print
                nn.AdaptiveAvgPool1d(1),
                nn.Softmax(dim=-1)
            ).to(self.device))

        self.critic = nn.Sequential(
            nn.Conv2d(self.C, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Flatten(),
            nn.Linear(621, 256),
            nn.ReLU(),
            nn.Linear(256, 1)
        ).to(self.device)

        self.optimizer = optim.Adam(self.parameters(), lr=0.001)

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.uniform_(m.weight, -0.1, 0.1)
                nn.init.uniform_(m.bias, -0.1, 0.1)




    def forward(self, state):
        state = state.view( self.C, self.H, self.W) # Reshape the input to (batch_size, C, H, W)
        print(f"Input state shape: {state.shape}")
        action_probs = [branch(state) for branch in self.actor_branches]
        for i, probs in enumerate(action_probs):
            print(f"Action probs shape at index {i}: {probs.shape}")
        value = self.critic(state)
        return action_probs, value

    def choose_action(self, state, mask, epsilon=0.5):
        state = torch.tensor(np.array([state]), dtype=torch.float).to(self.device)
        state = state.unsqueeze(0)  # Add an extra dimension for the batch size
        dist, _ = self.forward(state)
        action_probs = dist
        choosing_randomly = np.random.uniform() < epsilon
        # Call mask_multidiscrete_probs to mask the action_probs
        for i, probs in enumerate(action_probs):
            print(f"probs: {probs.shape} at index {i}")
        actions =  mask_apply(mask, action_probs, isRandom=choosing_randomly)
        # Choose actions based on the masked action probabilities
        # actions = [np.argmax(masked_action_probs[i]) for i in range(len(masked_action_probs))]
        return actions



    def train(self, transitions, gamma=0.99, critic_coeff=0.5, entropy_coeff=0.01):
        states, actions_list, rewards, next_states, masks, dones = zip(*transitions)

        actions = torch.tensor(actions_list, dtype=torch.long).to(self.device)

        states = torch.tensor(states, dtype=torch.float).to(self.device)
        next_states = torch.tensor(next_states, dtype=torch.float).to(self.device)

        rewards = torch.tensor(rewards, dtype=torch.float).to(self.device)
        dones = torch.tensor(dones, dtype=torch.float).to(self.device)

        # Calculate the discounted rewards
        discounted_rewards = []
        for i in range(len(rewards)):
            Gt = 0
            pw = 0
            for r in rewards[i:]:
                Gt = Gt + gamma**pw * r
                pw = pw + 1
            discounted_rewards.append(Gt)
        discounted_rewards = torch.tensor(discounted_rewards, dtype=torch.float).to(self.device)

        # Calculate the advantage
        _, critic_value = self.forward(states)
        _, next_critic_value = self.forward(next_states)
        advantage = discounted_rewards + gamma * next_critic_value.squeeze() * (1 - dones) - critic_value.squeeze()

        # Calculate actor (policy) loss
        action_probs, _ = self.forward(states)
        actor_loss = 0
        for i, action_prob in enumerate(action_probs):
            log_probs = torch.log(action_prob)
            log_probs = log_probs.gather(1, actions[:, i].unsqueeze(1))
            actor_loss += (-log_probs * advantage.detach()).mean()
        actor_loss /= len(action_probs)

        # Calculate critic loss
        _, critic_value = self.forward(states)
        critic_loss = 0.5 * advantage.pow(2).mean()

        # Calculate entropy loss
        action_probs, _ = self.forward(states)
        entropy = 0
        for action_prob in action_probs:
            entropy += (-action_prob * torch.log(action_prob)).mean()
        entropy_loss = -entropy_coeff * entropy

        # Total loss
        total_loss = actor_loss + critic_coeff * critic_loss + entropy_loss

        self.optimizer.zero_grad()
        total_loss.backward()
        self.optimizer.step()









In [ ]:
agent = ActorCritic(input_shape, num_actions)
print(num_actions)

In [ ]:
import logging

# increase batch size when we have a better model
batch_size = 1
num_epochs = 1000*15
max_episode_steps = 850

# log to file
LOG_FILENAME = 'training.log'
logging.basicConfig(filename=LOG_FILENAME,level=logging.DEBUG)
logging.info('Starting training')
#
# loaded = torch.load("model_68.pt")
# agent.load_state_dict(loaded)
# save transitions to csv

In [ ]:
import csv

In [ ]:

with tqdm(total=num_epochs*batch_size*max_episode_steps) as pbar:
    for epoch in range(num_epochs):
        episode_rewards = []
        print(f"Epoch: {epoch}")

        transitions = []

        for episode in range(batch_size):
            print(f"Episode: {episode}")
            obs = env.reset()
            state = preprocess_observation(obs)
            done = False
            episode_reward = 0

            for step in range(max_episode_steps):
                pbar.update(1)
                masks = obs["masks"]
                print("chooosing action, stoooooooooge!")
                action = agent.choose_action(state, masks)

                logging.info(f"action: {[i for i in action]}")
                print(f"action: {[i for i in action]}")
                logging.info(f"masks: {masks}")
                obs, reward, done, _ = env.step(action)
                base_reward = reward
                print(f"reward: {reward}")
                # entities, distances = obs["rays"]["entity_name"], obs["rays"]["entity_distance"]
                # sheep_idx = np.where(entities == "sheep")[0]
                #
                # if len(sheep_idx) > 0:
                #     sheep_distance = np.min(distances[sheep_idx])
                #     #encourage being closer to sheep
                #     if sheep_distance < 8:
                #         base_reward += 1 / (sheep_distance + 1)

                next_state = preprocess_observation(obs)

                episode_reward += base_reward
                transitions.append((state, action, reward, next_state, masks, done))

                if done:
                    pbar.update(max_episode_steps - step)
                    break

                state = next_state

            episode_rewards.append(episode_reward)

        # Train the agent using collected transitions
        agent.train(transitions)

        mean_episode_reward = np.mean(episode_rewards)
        torch.save(agent.state_dict(), f'model_{epoch}.pt')
        logging.info(f"Epoch: {epoch+1}/{num_epochs}, Mean Reward: {mean_episode_reward:.2f}, Epsilon: {0.5/(epoch+1)}")
        # print(f"Epoch: {epoch+1}/{num_epochs}, Mean Reward: {mean_episode_reward:.2f}")

# save the model
torch.save(agent.state_dict(), 'model.pt')